In [1]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import h5py
import tensorflow as tf
import matplotlib



import numpy as np
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils
from keras import backend as K
from sklearn.model_selection import StratifiedKFold

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype s

In [2]:
# initialize the number of epochs to train for, initial learning rate,
# and batch size
dataset = 'data'
# model_path = 'AllStressesModel_upsdataample_dropout.h5'
plot = 'Accuracy_plot_upsample_dropout'

In [3]:
# read in data ################################################################
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []
 
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images(dataset)))
random.seed(42)
random.shuffle(imagePaths)

[INFO] loading images...


In [4]:
# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)
    
    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    if label == 'rose':
        label = 0
    elif label == 'daisy':
        label = 1
    elif label == 'tulip':
        label = 2
    elif label == 'sunflower':
        label = 3
    elif label == 'dandelion':
        label = 4
#     elif label == 'PhosphorusDeficiency':
#         label = 5
#     elif label == 'Rust':
#         label = 6
    labels.append(label)

In [5]:
len(data)

4323

In [6]:
len(labels)

4323

In [7]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [8]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(28, 28,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5))
    model.add(Activation('softmax'))

    return model
###############################################################################
# train the network



In [9]:

EPOCHS = 15
INIT_LR = 1e-3
BS = 32

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)

# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=5)
testY = to_categorical(testY, num_classes=5)

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

# # initialize the model
# print("[INFO] compiling model...")
# model = create_model()
# opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# model.compile(loss="categorical_crossentropy", optimizer=opt,
#     metrics=["accuracy"])

# # train the network
# print("[INFO] training network...")
# H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
#     validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
#     epochs=EPOCHS, verbose=1)

# # save the model to disk
# print("[INFO] serializing network...")
# model.save("model.h5")

# # plot the training loss and accuracy
# plt.style.use("ggplot")
# plt.figure()
# N = EPOCHS
# plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
# plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
# plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
# plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
# plt.title("Training Loss and Accuracy on All Stresses")
# plt.xlabel("Epoch #")
# plt.ylabel("Loss/Accuracy")
# plt.legend(loc="lower left")
# plt.savefig(plot)

In [10]:
# calculate confusion matrix ##################################################
import pandas as pd
model = create_model()
y_pred = model.predict(testX)
y_true = testY.copy()

y_pred = pd.DataFrame(y_pred)
y_true = pd.DataFrame(y_true)

y_pred['class'] = y_pred.idxmax(axis=1)
y_true['class'] = y_true.idxmax(axis=1)

confusion_matrix(y_true['class'],y_pred['class'])

# cross validation ############################################################
from sklearn.model_selection import StratifiedKFold
X = data
Y = labels
seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = seed)
cvscores = []
for train, test in kfold.split(X,Y):
    
    # convert the labels from integers to vectors
    trainY = to_categorical(Y[train], num_classes=5)
    testY = to_categorical(Y[test], num_classes=5)
    
    # construct the image generator for data augmentation
    aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")
    print("[INFO] compiling model...")
    
    model2 = create_model()
    
    opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
    model2.compile(loss="categorical_crossentropy", optimizer=opt,
                  metrics=["accuracy"])
    
    # train the network
    print("[INFO] training network...")
    model2.fit_generator(aug.flow(X[train], trainY, batch_size=BS),
                            steps_per_epoch=len(X[train]) // BS,
                            epochs=EPOCHS, verbose=1)
    
    scores = model2.evaluate(X[test],testY, verbose = 1)
    print("%s: %.2f%%" % (model2.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("Done!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

[INFO] compiling model...
[INFO] training network...
Epoch 1/15
121/121 [==============================] - 2s 15ms/step - loss: 1.5436 - acc: 0.2792
Epoch 2/15
121/121 [==============================] - 2s 15ms/step - loss: 1.2981 - acc: 0.4199
Epoch 3/15
121/121 [==============================] - 2s 15ms/step - loss: 1.2362 - acc: 0.4370
Epoch 4/15
121/121 [==============================] - 2s 14ms/step - loss: 1.2196 - acc: 0.4659
Epoch 5/15
121/121 [==============================] - 2s 14ms/step - loss: 1.1605 - acc: 0.5031
Epoch 6/15
121/121 [==============================] - 2s 15ms/step - loss: 1.1224 - acc: 0.5374
Epoch 7/15
121/121 [==============================] - 2s 15ms/step - loss: 1.0714 - acc: 0.5597
Epoch 8/15
121/121 [==============================] - 2s 15ms/step - loss: 1.0318 - acc: 0.5925
Epoch 9/15
121/121 [==============================] - 2s 15ms/step - loss: 1.0194 - acc: 0.5834
Epoch 10/15
121/121 [==============================] - 2s 15ms/step - loss: 1.0137 

121/121 [==============================] - 2s 15ms/step - loss: 1.1458 - acc: 0.5168
Epoch 5/15
121/121 [==============================] - 2s 15ms/step - loss: 1.1192 - acc: 0.5361
Epoch 6/15
121/121 [==============================] - 2s 15ms/step - loss: 1.0880 - acc: 0.5491
Epoch 7/15
121/121 [==============================] - 2s 14ms/step - loss: 1.0524 - acc: 0.5745
Epoch 8/15
121/121 [==============================] - 2s 14ms/step - loss: 1.0399 - acc: 0.5882
Epoch 9/15
121/121 [==============================] - 2s 15ms/step - loss: 1.0026 - acc: 0.5991
Epoch 10/15
121/121 [==============================] - 2s 15ms/step - loss: 0.9920 - acc: 0.6101
Epoch 11/15
121/121 [==============================] - 2s 15ms/step - loss: 0.9961 - acc: 0.6051
Epoch 12/15
121/121 [==============================] - 2s 15ms/step - loss: 0.9825 - acc: 0.6095
Epoch 13/15
121/121 [==============================] - 2s 15ms/step - loss: 0.9788 - acc: 0.6138
Epoch 14/15
121/121 [==========================